In [4]:
import numpy as np
from scipy.sparse import coo_matrix, hstack 
from mip import Model, xsum, minimize, BINARY, OptimizationStatus
from bposd.css import css_code
from ldpc import bposd_decoder, mod2
import pickle
import itertools

import sys
import os
sys.path.append(os.path.abspath("..")) 

from functions_BB_code import ell, m, n, n2, A, B, AT, BT, num_end 
from functions_BB_code import distance_test, rank2, get_connection_Tanner, get_SM_circuit_parallel
from functions_BB_code import get_Set_noisy_circuits_logical_X, get_Set_noisy_circuits_logical_Z
from functions_BB_code import decoding_X_matrix, decoding_Z_matrix
from functions_BB_code import simulate_syndrome_Lx, simulate_syndrome_Lz 

from scipy.io import savemat
from scipy.io import loadmat

## Code construction

In [12]:
# [[18, 4, 4]]
ell,m = 3, 3
a1,a2,a3 = 1, 0, 2
b1,b2,b3 = 1, 0, 2

# code length
n = 2*m*ell;  n2 = m*ell

I_ell = np.identity(ell,dtype=int) ; I_m = np.identity(m,dtype=int); I = np.identity(ell*m,dtype=int)
x = {} ;  y = {}

for i in range(ell):
	x[i] = np.kron(np.roll(I_ell,i,axis=1),I_m)
for i in range(m):
	y[i] = np.kron(I_ell,np.roll(I_m,i,axis=1))

A = (x[a1%ell] + y[a2%m] + y[a3%m]) % 2
B = (y[b1%m] + x[b2%ell] + x[b3%ell]) % 2

A1 = x[a1%ell]; A2 = y[a2%m]; A3 = y[a3%m]
B1 = y[b1%m]; B2 = x[b2%ell]; B3 = x[b3%ell]

AT = np.transpose(A) ; BT = np.transpose(B)

# Testing CSS code
hx = np.hstack((A,B));  hz = np.hstack((BT,AT))

remove_X_list = [2,8] ;  remove_Z_list = [3,4] ;

hx = np.delete(hx, remove_X_list, axis=0) ; hz = np.delete(hz, remove_Z_list, axis=0)

# number of logical qubits
k = n - rank2(hx) - rank2(hz)

qcode=css_code(hx=hx, hz=hz)

# logical operator
lz = qcode.lz ;  lx = qcode.lx
# we choose a set of basis such that each logical X anticommute with its corresponding logical Z.
lz[1] = (lz[1]+lz[2]) %2 ; lz[3] = (lz[3]+lz[0]) %2

lz_copy = lz.copy()
lz_copy[0] = lz[1] ;  lz_copy[1] = lz[0] ;  lz_copy[2] = lz[3] ;  lz_copy[3] = lz[2] ;
lz = lz_copy
(lx@lz.T) %2

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [6]:
# the order of two-qubit gates
sX = ['idle', 'idle', 1, 4, 3, 5, 0, 2] ; sZ = ['idle', 3, 5, 0, 1, 2, 4, 'idle'] ;
##------------------------------------------------------------------------------------------------------------

# Connections of edges in the Tanner graph
lin_order, data_qubits, Xchecks, Zchecks, nbs = get_connection_Tanner(remove_X_list, remove_Z_list, n2)

SM_cycle = get_SM_circuit_parallel(remove_X_list, remove_Z_list, lin_order, data_qubits, Xchecks, Zchecks, nbs, sX, sZ ) ;

cycle_append = []
for q in data_qubits:
    cycle_append.append(('final',q))

In [9]:
# number of syndrome measurement cycles 
num_cycles = 1
from component_error_rates import *

In [10]:
# full syndrome measurement circuit
cycle_repeated = (num_cycles-1) * SM_cycle[2*n2-len(remove_X_list)-len(remove_Z_list):] + \
            SM_cycle[2*n2-len(remove_X_list)-len(remove_Z_list) : -2*n2] + cycle_append  ;

# Generating noisy circuits with a singe faulty operation
X_circuits, X_Prob = get_Set_noisy_circuits_logical_X(cycle_repeated, error_rate_init, error_rate_idle, error_rate_H, error_rate_cz, \
                                                      error_rate_meas, error_final, error_DD_phaseflip, error_DD_bitflip)
num_errX=len(X_circuits)
print('Number of noisy circuits for the logical X state =',num_errX)

channel_probsX, HX, HdecX, HXdict = decoding_X_matrix(X_circuits, X_Prob, num_cycles, SM_cycle, lin_order, n, k, data_qubits, \
                                                      Xchecks, lx, remove_X_list, remove_Z_list) ;

Z_circuits, Z_Prob = get_Set_noisy_circuits_logical_Z(cycle_repeated, error_rate_init, error_rate_idle, error_rate_H, error_rate_cz, \
                                                      error_rate_meas, error_final, error_DD_phaseflip, error_DD_bitflip)
num_errZ=len(Z_circuits)
print('Number of noisy circuits for the logical Z state =',num_errZ)    

channel_probsZ, HZ, HdecZ, HZdict = decoding_Z_matrix(Z_circuits, Z_Prob, num_cycles, SM_cycle, lin_order, n, k, data_qubits, \
                                                          Zchecks, lz, remove_X_list, remove_Z_list) ;

Number of noisy circuits for the logical X state = 1664
Number of distinct syndrome histories for logical X = 89
Number of noisy circuits for the logical Z state = 1664
Number of distinct syndrome histories for logical Z = 89


In [18]:
HZ = HZ.todense()
HX = HX.todense()
HdecZ = HdecZ.todense()
HdecX = HdecX.todense()

## Using optimization algorithm to calculate the circuit-level distance

In [19]:
# circuit-level distance for X-type errors
D_circuit_x = 4
for i in range(1,k+1):
    d = distance_test(np.array(HdecZ), np.array( HZ[-i,:])[0]  )
    D_circuit_x = min(D_circuit_x, d)

# circuit-level distance for Z-type errors
D_circuit_z = 4
for i in range(1,k+1):
    d = distance_test(np.array(HdecX), np.array( HX[-i,:])[0]  )
    D_circuit_z = min(D_circuit_z, d)

D_circuit = min(D_circuit_x, D_circuit_z)
print(f"The circuit-level distance of the [[18,4,4]] BB code is:{D_circuit}")

The circuit-level distance of the [[18,4,4]] BB code is:3


## Using the BP-OSD algorithm to calculate the upper bound of the circuit-level distance 
(the code is from https://github.com/sbravyi/BivariateBicycleCodes/blob/main/circuit_distance.py)

In [22]:
# number of Monte Carlo trials
num_trials = 500

# error rate 
p = 0.003

In [23]:
# setup BP-OSD decoder parameters
my_bp_method = "ms"
my_max_iter = 1000
my_osd_method = "osd_cs"
my_osd_order = 7
my_ms_scaling_factor = 0

# stores the minimum weight of logical X and Z operators
wminX = HdecX.shape[1]
wminZ = HdecZ.shape[1]

for trial in range(num_trials):

	ec_resultZ = 0
	ec_resultX = 0
	
	# correct Z errors
	random_vector = np.random.randint(2,size=HZ.shape[0]) 
	random_logical_op = (random_vector @ HZ) % 2
	random_logical_op = np.reshape(random_logical_op, (1,HZ.shape[1]))
	HZ1 = np.vstack((HdecZ,random_logical_op))
	syndrome = np.zeros(HZ1.shape[0],dtype=int)
	syndrome[-1]=1

	bpdZ=bposd_decoder(
   		HZ1,
    	error_rate=0.01,
	    max_iter=my_max_iter, #the maximum number of iterations for BP)
	    bp_method=my_bp_method,
	    ms_scaling_factor=my_ms_scaling_factor, #min sum scaling factor. If set to zero the variable scaling factor method is used
	    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
	    osd_order=my_osd_order #the osd search depth
	    )
	bpdZ.decode(syndrome)
	low_weight_logical = bpdZ.osdw_decoding
	wt = np.count_nonzero(low_weight_logical)
	if wt<wminZ and wt>0:
		wminZ = wt
	print('Logical Z weight =',wt,'minimum Z weight =',wminZ)

		
	# correct X errors 
	random_vector = np.random.randint(2,size=HX.shape[0]) 
	random_logical_op = (random_vector @ HX) % 2
	random_logical_op = np.reshape(random_logical_op,(1,HX.shape[1]))
	HX1 = np.vstack((HdecX,random_logical_op))
	syndrome = np.zeros(HX1.shape[0],dtype=int)
	syndrome[-1]=1

	bpdX=bposd_decoder(
   		HX1,
    	error_rate=0.01,
	    max_iter=my_max_iter, #the maximum number of iterations for BP)
	    bp_method=my_bp_method,
	    ms_scaling_factor=my_ms_scaling_factor, #min sum scaling factor. If set to zero the variable scaling factor method is used
	    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
	    osd_order=my_osd_order #the osd search depth
	    )
	bpdX.decode(syndrome)
	low_weight_logical = bpdX.osdw_decoding
	wt = np.count_nonzero(low_weight_logical)
	if wt<wminX and wt>0:
		wminX = wt
	print('Logical X weight =',wt,'minimum X weight =',wminX)

Logical Z weight = 4 minimum Z weight = 4
Logical X weight = 3 minimum X weight = 3
Logical Z weight = 0 minimum Z weight = 4
Logical X weight = 4 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 4 minimum X weight = 3
Logical Z weight = 4 minimum Z weight = 3
Logical X weight = 9 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 3 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 4 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 4 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 3 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 4 minimum X weight = 3
Logical Z weight = 4 minimum Z weight = 3
Logical X weight = 3 minimum X weight = 3
Logical Z weight = 4 minimum Z weight = 3
Logical X weight = 4 minimum X weight = 3
Logical Z weight = 3 minimum Z weight = 3
Logical X weight = 3 minimum X wei